In [1]:
!pip install xgboost
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install -U scikit-learn

In [1]:
#Import libraries
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [3]:
#Import Dataset + Data Preprocessing
df_mitbih_train = pd.read_csv(r"C:\Users\calvi\Documents\Datasets\ecg_dataset\mitbih_train.csv", header = None)
df_mitbih_test = pd.read_csv(r"C:\Users\calvi\Documents\Datasets\ecg_dataset\mitbih_test.csv", header = None)

df_mitbih_train.shape, df_mitbih_test.shape

((87554, 188), (21892, 188))

In [4]:
X_train = df_mitbih_train.drop(columns=[187])
y_train = df_mitbih_train[187]
X_test = df_mitbih_test.drop(columns=[187])
y_test = df_mitbih_test[187]

In [5]:
model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, objective = 'multi:softmax', n_jobs = -1, num_class = 5, learning_rate = 0.1)
model.fit(X_train, y_train)
print(); print(model)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_class=5, num_parallel_tree=None,
              objective='multi:softmax', ...)


In [6]:
expected_y  = y_test
predicted_y = model.predict(X_test)

print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99     18118
         1.0       0.94      0.61      0.74       556
         2.0       0.97      0.89      0.92      1448
         3.0       0.86      0.69      0.77       162
         4.0       0.99      0.95      0.97      1608

    accuracy                           0.97     21892
   macro avg       0.95      0.83      0.88     21892
weighted avg       0.97      0.97      0.97     21892



In [26]:
n_estimators = range(3000, 3101, 100)
param_grid = dict(n_estimators=n_estimators)

grid_cv = GridSearchCV(model, param_grid, return_train_score = True, verbose = 3)
_ = grid_cv.fit(X_test, y_test)

print(f'Best Parameters:', grid_cv.best_params_)
print(f'Best Score:', grid_cv.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END n_estimators=3000;, score=(train=1.000, test=0.977) total time=  37.6s
[CV 2/5] END n_estimators=3000;, score=(train=1.000, test=0.969) total time=  37.5s
[CV 3/5] END n_estimators=3000;, score=(train=1.000, test=0.974) total time=  37.7s
[CV 4/5] END n_estimators=3000;, score=(train=1.000, test=0.978) total time=  38.2s
[CV 5/5] END n_estimators=3000;, score=(train=1.000, test=0.972) total time=  37.6s
[CV 1/5] END n_estimators=3100;, score=(train=1.000, test=0.977) total time=  38.5s
[CV 2/5] END n_estimators=3100;, score=(train=1.000, test=0.969) total time=  38.2s
[CV 3/5] END n_estimators=3100;, score=(train=1.000, test=0.974) total time=  38.5s
[CV 4/5] END n_estimators=3100;, score=(train=1.000, test=0.978) total time=  38.5s
[CV 5/5] END n_estimators=3100;, score=(train=1.000, test=0.972) total time=  38.2s
Best Parameters: {'n_estimators': 3000}
Best Score: 0.974054530160821


In [63]:
!pip install hyperopt

from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import warnings
warnings.simplefilter("ignore")

In [59]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 3000,
        'seed': 0
    }

In [60]:
clf = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, objective = 'multi:softmax', n_jobs = -1, num_class = 5, learning_rate = 0.3)
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators = 3000, max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [61]:
print(); print(clf)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_class=5, num_parallel_tree=None,
              objective='multi:softmax', ...)


In [64]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                 
0.8332724282843048                                     
SCORE:                                                                            
0.8336378585784762                                                                
SCORE:                                                                            
0.8338662525123333                                                                
SCORE:                                                                              
0.8336378585784762                                                                  
SCORE:                                                                              
0.8338662525123333                                                                  
SCORE:                                                                              
0.8329983555636762                                                                  
SCORE:                                        

In [65]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.934397209208678, 'gamma': 2.056550422157872, 'max_depth': 6.0, 'min_child_weight': 6.0, 'reg_alpha': 41.0, 'reg_lambda': 0.251398330291195}


In [8]:
last_model = xgb.XGBClassifier(
                    n_estimators = 3000,
                    learning_rate = 0.1,
                    max_depth = 6,
                    gamma = 2.056550422157872,
                    reg_alpha = 41,
                    min_child_weight= 6,
                    colsample_bytree= 0.934397209208678,
                    reg_lambda= 0.251398330291195,
                    tree_method='gpu_hist',
                    gpu_id=0,
                    objective = 'multi:softmax',
                    n_jobs = -1,
                    num_class = 5
)

last_model.fit(X_train, y_train)
print(); print(last_model)
expected_y  = y_test
predicted_y = last_model.predict(X_test)
print(metrics.classification_report(expected_y, predicted_y))


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.934397209208678, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.056550422157872, gpu_id=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=6, missing=nan,
              monotone_constraints=None, n_estimators=3000, n_jobs=-1,
              num_class=5, num_parallel_tree=None, objective='multi:softmax', ...)
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     18118
         1.0       0.96      0.56      0.71       556
         2.0       0.95      0.85      0.90      1448
         3